## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [ ]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [2]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]




In [4]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime',
            'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion',
            'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes



Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split



In [8]:
# Convert 'Education' to numeric using map()
education_mapping = {
    'Below College': 0,
    'College': 1,
    'Bachelor': 2,
    'Master': 3,
    'Doctor': 4
}
education_numeric = X_df['Education'].map(education_mapping)

# Convert 'OverTime' to numeric using map()
overtime_mapping = {'Yes': 1, 'No': 0}
overtime_numeric = X_df['OverTime'].map(overtime_mapping)

# Create a new DataFrame with the updated columns
X_df = pd.DataFrame({
    'Education': education_numeric,
    'Age': X_df['Age'],
    'DistanceFromHome': X_df['DistanceFromHome'],
    'JobSatisfaction': X_df['JobSatisfaction'],
    'OverTime': overtime_numeric,
    'StockOptionLevel': X_df['StockOptionLevel'],
    'WorkLifeBalance': X_df['WorkLifeBalance'],
    'YearsAtCompany': X_df['YearsAtCompany'],
    'YearsSinceLastPromotion': X_df['YearsSinceLastPromotion'],
    'NumCompaniesWorked': X_df['NumCompaniesWorked']
})

# Print the updated data types
X_df.dtypes


Education                  float64
Age                          int64
DistanceFromHome             int64
JobSatisfaction              int64
OverTime                   float64
StockOptionLevel             int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
NumCompaniesWorked           int64
dtype: object

In [15]:
# Convert DataFrame to NumPy array if not already done
if not isinstance(X_train, np.ndarray):
    X_train = X_train.to_numpy()
if not isinstance(X_test, np.ndarray):
    X_test = X_test.to_numpy()

# Handle missing values and invalid values
X_train = X_train.astype(float)
X_test = X_test.astype(float)

# Replace NaN values with 0
X_train[np.isnan(X_train)] = 0
X_test[np.isnan(X_test)] = 0

# Replace infinite values with large finite values
X_train[np.isinf(X_train)] = np.finfo(X_train.dtype).max * np.sign(X_train[np.isinf(X_train)])
X_test[np.isinf(X_test)] = np.finfo(X_test.dtype).max * np.sign(X_test[np.isinf(X_test)])

# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training data
X_train_scaled = X_scaler.transform(X_train)

# Scale the testing data
X_test_scaled = X_scaler.transform(X_test)




In [18]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder to the training and testing data
y_train_department_encoded = department_encoder.transform(y_train[['Department']])
y_test_department_encoded = department_encoder.transform(y_test[['Department']])



In [19]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])



## Create, Compile, and Train the Model

In [20]:
from tensorflow.keras.layers import Input, Dense

# Find the number of columns in the X training data
num_columns = X_train_scaled.shape[1]

# Create the input layer
input_layer = Input(shape=(num_columns,))

# Create at least two shared layers
shared_layer1 = Dense(64, activation='relu')(input_layer)
shared_layer2 = Dense(32, activation='relu')(shared_layer1)


In [21]:
from tensorflow.keras.layers import Dense

# Create a branch for Department with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = Dense(16, activation='relu')(shared_layer2)

# Create the output layer
department_output_layer = Dense(y_train_department_encoded.shape[1], activation='softmax', name='department_output')(department_hidden_layer)



In [22]:
from tensorflow.keras.layers import Dense

# Create a branch for Attrition with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = Dense(16, activation='relu')(shared_layer2)

# Create the output layer
attrition_output_layer = Dense(y_train_attrition_encoded.shape[1], activation='sigmoid', name='attrition_output')(attrition_hidden_layer)



In [23]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 64)                   704       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 32)                   2080      ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 16)                   528       ['dense_1[0][0]']             
                                                                                              

In [24]:
# Train the model
history = model.fit(
    X_train_scaled,
    {'department_output': y_train_department_encoded, 'attrition_output': y_train_attrition_encoded},
    epochs=100,
    batch_size=32,
    verbose=1
)



Epoch 1/100
37/37 [==============================] - 2s 3ms/step - loss: 1.6634 - department_output_loss: 1.0480 - attrition_output_loss: 0.6154 - department_output_accuracy: 0.5179 - attrition_output_accuracy: 0.7789
Epoch 2/100
37/37 [==============================] - 0s 2ms/step - loss: 1.3304 - department_output_loss: 0.8363 - attrition_output_loss: 0.4941 - department_output_accuracy: 0.6505 - attrition_output_accuracy: 0.8316
Epoch 3/100
37/37 [==============================] - 0s 3ms/step - loss: 1.2434 - department_output_loss: 0.7895 - attrition_output_loss: 0.4540 - department_output_accuracy: 0.6505 - attrition_output_accuracy: 0.8316
Epoch 4/100
37/37 [==============================] - 0s 2ms/step - loss: 1.2108 - department_output_loss: 0.7775 - attrition_output_loss: 0.4333 - department_output_accuracy: 0.6505 - attrition_output_accuracy: 0.8316
Epoch 5/100
37/37 [==============================] - 0s 2ms/step - loss: 1.1910 - department_output_loss: 0.7710 - attrition_out

In [25]:
# Evaluate the model with the testing data
loss, department_loss, attrition_loss, department_accuracy, attrition_accuracy = model.evaluate(
    X_test_scaled,
    {'department_output': y_test_department_encoded, 'attrition_output': y_test_attrition_encoded},
    verbose=1
)

10/10 [==============================] - 2s 7ms/step - loss: 1.5075 - department_output_loss: 1.0228 - attrition_output_loss: 0.4846 - department_output_accuracy: 0.5986 - attrition_output_accuracy: 0.8027


In [26]:
# Print the accuracy for both department and attrition
print("Department predictions accuracy:", department_accuracy)
print("Attrition predictions accuracy:", attrition_accuracy)

Department predictions accuracy: 0.5986394286155701
Attrition predictions accuracy: 0.8027210831642151


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is not always the best metric to use for evaluating a model's performance, especially in certain scenarios. You could have Class imbalance which would throw off your results.
2. I chose different activation functions for the output layers of the Department and Attrition branches.
3. The model could be improved with feature selection and engineering or hyperparameter tuning.